# Training language models to follow instructions with human feedback

```{note}
In this paper, we show an avenue for
aligning language models with user intent on a wide range of tasks by fine-tuning
with human feedback.
```

## Introduction

The language modeling objective used for many recent large LMs—predicting the next token on a webpage from the internet—is different from the objective “follow the user’s instructions helpfully and safely”.



Large language models (LMs) can be “prompted” to perform a range of natural language processing (NLP) tasks, given some examples of the task as input. However, these models often express unintended behaviors such as making up facts, generating biased or toxic text, or simply not following user instructions. This is because the language modeling objective used for many recent large LMs—predicting the next token on a webpage from the internet—is different from the objective “follow the user’s instructions helpfully and safely”. Thus, we say that the language modeling objective is *misaligned*.

We focus on fine-tuning approaches to aligning language models.
* First hire a team of 40 contractors to label our data, then collect a dataset of human-written demonstrations of the desired output behavior on (mostly English) prompts submitted to the OpenAI API3 and some labeler-written prompts, and use this to train our supervised learning baselines.

* Next, we collect a dataset of human-labeled comparisons between outputs from our models on a larger set of API prompts. We then train a reward model (RM) on this dataset to predict which model output our labelers would prefer.

* Finally, we use this RM as a reward function and fine-tune our supervised learning baseline to maximize this reward using the PPO algorithm.

This procedure aligns the behavior of GPT-3 to the stated preferences of a specific group of people (mostly our labelers and researchers), we call the resulting models *InstructGPT*.

We mainly evaluate our models by having our labelers rate the quality of model outputs on our test set, consisting of prompts from held-out customers (who are not represented in the training data).

![](images/instruct1.png)

## Methods and experimental details

### High-level methodology

We start with a pretrained language model, a distribution of prompts on which we want our model to produce aligned outputs, and a team of trained human labelers. We then apply the following three steps:

**Step 1: Collect demonstration data, and train a supervised policy.**

**Step 2: Collect comparison data, and train a reward model.**

**Step 3: Optimize a policy against the reward model using PPO.**

Steps 2 and 3 can be iterated continuously; more comparison data is collected on the current best policy, which is used to train a new RM and then a new policy.

### Dataset

Our prompt dataset consists of text prompts submitted to the OpenAI API and prompts written by labelers. From these prompts, we produce three different datasets used in our fine-tuning procedure:

1. SFT dataset, with labeler demonstrations used to train our SFT models, contains about 13k training prompts (from the API and labeler-written)
2. RM dataset, with labeler rankings of model outputs used to train our RMs, has 33k training prompts (from the API and labeler-written)
3. PPO dataset, without any human labels, which are used as inputs for RLHF fine-tuning, has 31k training prompts (only from the API)

### Models

**Supervised fine-tuning (SFT).** We fine-tune GPT-3 on our labeler demonstrations using supervised
learning.

**Reward modeling (RM).** Starting from the SFT model with the final unembedding layer removed, we trained a model to take in a prompt and response, and output a scalar reward.

In order to speed up comparison collection, we present labelers with anywhere between $K=4$ and $K = 9$ responses to rank, this produce ${k \choose 2}$ comparisons for each prompt shown to a labeler. we train on all ${k \choose 2}$ comparisons from each prompt as a single batch element.

The loss function for the reward model is:

$$\mbox{loss}(\theta) = -\frac{1}{{k\choose 2}}\mathbb{E}_{(x, y_{w}, y_{l})\sim{D}}[\log(\sigma(r_{\theta}(x, y_{w}) - r_{\theta}(x, y_{l})))]$$

where $r_{\theta}(x, y)$ is the scalar output of the reward model for prompt $x$ and completion $y$ with parameters $\theta$, $y_{w}$ is the preferred completion out of the pair of $y_{w}$ and $y_{l}$, and $D$ is the dataset of human comparisons. 

Finally, since the RM loss is invariant to shifts in reward, we normalize the reward model using a bias so that the labeler demonstrations achieve a mean score of 0 before doing RL.

**Reinforcement learning (RL).** We fine-tuned the SFT model on our environment using PPO. The environment presents a random customer prompt and expects a response to the prompt, given the prompt and response, it produces a reward determined by the reward model and ends the episode. In addition, we add a KL penalty from the SFT model to mitigate over-optimization of the reward model. We call these
models “PPO.”

We also experiment with mixing the pretraining gradients into the PPO gradients, in order to fix the performance regressions on public NLP datasets. We call these models “PPO-ptx.” We maximize the following combined objective function in RL training:

$$
\begin{aligned}
\text{objective}(\phi) =& \mathbb{E}_{(x,y)\sim D_{\pi_{\phi}^{\text{RL}}}}\left[r_{\theta}(x, y) - \beta\log\left(\pi_{\phi}^{\text{RL}}(y|x)/\pi^{\text{SFT}}(y|x)\right)\right] + \\
&\gamma\mathbb{E}_{x\sim D_{\text{pretrain}}}\left[\log(\pi_{\phi}^{\text{RL}}(x))\right]
\end{aligned}
$$

where $\pi_{\phi}^{\text{RL}}$ is the learned RL policy, $\pi^{\text{SFT}}$ is the supervised trained model, and $D_{\text{pretrain}}$ is the pretraining distribution. The KL reward coefficient, $\beta$, and the pretraining loss coefficient, $\gamma$, control
the strength of the KL penalty and pretraining gradients respectively.

### Evaluation

To evaluate how “aligned” our models are, we first need to clarify what alignment means in this context. We use the definition that models to be aligned if they are helpful, honest, and harmless.

To be helpful, the model should follow instructions, but also infer intention from a few-shot prompt or another interpretable pattern such as “Q: {question}\nA:”. Since a given prompt’s intention can be unclear or ambiguous, we rely on judgment from our labelers, and our main metric is labeler preference ratings.

It is unclear how to measure honesty in purely generative models, Instead, we measure truthfulness—whether the model’s statements about the world are true—using two metrics: (1) evaluating our model’s tendency to make up information on closed domain tasks (“hallucinations”), and (2) using the TruthfulQA dataset.

Similarly to honesty, measuring the harms of language models also poses many challenges. we have labelers evaluate whether an output is inappropriate in the context of a customer assistant, denigrates a protected class, or contains sexual or violent content. We also benchmark our model on datasets intended to measure bias and toxicity, such as RealToxicityPrompts.

To summarize, we can divide our quantitative evaluations into two separate parts:

**Evaluations on API distribution.** Our main metric is human preference ratings on a held out set of prompts from the same source as our training distribution.

**Evaluations on public NLP datasets.** We evaluate on two types of public datasets: those that capture an aspect of language model safety, particularly truthfulness, toxicity, and bias, and those that capture zero-shot performance on traditional NLP tasks like question answering, reading comprehension, and summarization.